In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import holidays
from sklearn.preprocessing import LabelEncoder
from kmodes.kprototypes import KPrototypes

### Select reasonable date range

In [4]:
# Read in raw data
df = pd.read_csv('data/Major_Crime_Indicators_Open_Data.csv', parse_dates=['OCC_DATE'])

In [5]:
# Find min and max occurence dates
df.OCC_DATE.min(), df.OCC_DATE.max()

(Timestamp('1966-06-09 04:00:00+0000', tz='UTC'),
 Timestamp('2022-12-31 05:00:00+0000', tz='UTC'))

In [6]:
# Find min and max report dates
df.REPORT_DATE.min(), df.REPORT_DATE.max()

('2014/01/01 05:00:00+00', '2022/12/31 05:00:00+00')

We select crime occurences after the year 2014 (corresponding to the earliest report date in the dataset). In this analysis, we focus on occurence date and not report date, since it does not make sense for a user to know when a hypothetical future crime will be reported, but the day, month, and time of a crime occurring may be important predictors.

In [7]:
df = df[df['OCC_DATE'] >= '2014/01/01']

In [8]:
# check work
df.OCC_DATE.min(), df.OCC_DATE.max()

(Timestamp('2014-01-01 05:00:00+0000', tz='UTC'),
 Timestamp('2022-12-31 05:00:00+0000', tz='UTC'))

### Select columns of interest
We select columns relating to the location and time reported crimes have occured. Redundant variables are dropped. The goal is to use these predictors to predict crime type (MCI_CATEGORY)

In [9]:
df = df[['OBJECTID', 'OCC_DATE', 'OCC_HOUR', 'LOCATION_TYPE', 'NEIGHBOURHOOD_158', 'LONG_WGS84', 'LAT_WGS84', 'MCI_CATEGORY']].copy()

### Missing values
The data does not appear to have missing values.

In [10]:
df.isna().sum(axis=0)

OBJECTID             0
OCC_DATE             0
OCC_HOUR             0
LOCATION_TYPE        0
NEIGHBOURHOOD_158    0
LONG_WGS84           0
LAT_WGS84            0
MCI_CATEGORY         0
dtype: int64

### Process names of location type
Delete the parenthesis and everything inside using regex

In [11]:
df.loc[:, 'LOCATION_TYPE'] = df.LOCATION_TYPE.str.replace('\s*\(.+', '', regex=True)

In [12]:
df.LOCATION_TYPE.unique()

array(['Commercial Dwelling Unit', 'Apartment',
       'Streets, Roads, Highways', 'Bar / Restaurant',
       'Other Commercial / Corporate Places', 'Go Train',
       'Single Home, House',
       'Hospital / Institutions / Medical Facilities', 'Retirement Home',
       'Gas Station', 'Open Areas', 'Parking Lots', 'Ttc Subway Station',
       'Other Non Commercial / Corporate Places', 'Unknown',
       'Bank And Other Financial Institutions', 'Convenience Stores',
       'Jails / Detention Centres', 'Group Homes',
       'Private Property Structure', 'Homeless Shelter / Mission',
       'Ttc Street Car', 'Ttc Subway Train',
       'Schools During Un-Supervised Activity', 'Construction Site',
       'Police / Courts', 'Ttc Bus', 'Schools During Supervised Activity',
       'Religious Facilities', 'Universities / Colleges', 'Dealership',
       'Ttc Bus Stop / Shelter / Loop', 'Go Station',
       'Other Passenger Train', 'Other Regional Transit System Vehicle',
       'Other Passenger T

We notice that many of the public transit categories are similar and can be binned together to reduce the cardinality of the location variable

In [13]:
def group_transit(string):
    '''For a given string, if it belongs in the public transit category, return 'Public Transit'''
    
    public_transit_elements = ['Ttc Bus', 'Ttc Subway Train', 'Ttc Subway Tunnel / Outdoor Tracks', 
                               'Ttc Street Car', 'Ttc Support Vehicle', 'Ttc Bus Garage', 
                               'Ttc Light Rail Vehicle', 'Ttc Light Rail Transit Station', 
                               'Ttc Wheel Trans Vehicle', 'Ttc Bus Stop / Shelter / Loop', 
                               'Go Train', 'Go Station', 'Go Bus', 'Other Passenger Train', 
                               'Other Regional Transit System Vehicle', 'Other Passenger Train Station', 
                               'Other Train Tracks', 'Other Train Admin Or Support Facility', 
                               'Cargo Train', 'Other Train Yard']
    
    if string in public_transit_elements:
        return 'Public Transit'
    else:
        return string

In [14]:
# Use pandas function map to apply group_transit() to every element in LOCATION_TYPE
df.loc[:, 'LOCATION_TYPE'] = df.LOCATION_TYPE.map(group_transit)

In [15]:
# We obtain 33 different location types
len(df.LOCATION_TYPE.unique())

33

In [16]:
df.LOCATION_TYPE.unique()

array(['Commercial Dwelling Unit', 'Apartment',
       'Streets, Roads, Highways', 'Bar / Restaurant',
       'Other Commercial / Corporate Places', 'Public Transit',
       'Single Home, House',
       'Hospital / Institutions / Medical Facilities', 'Retirement Home',
       'Gas Station', 'Open Areas', 'Parking Lots', 'Ttc Subway Station',
       'Other Non Commercial / Corporate Places', 'Unknown',
       'Bank And Other Financial Institutions', 'Convenience Stores',
       'Jails / Detention Centres', 'Group Homes',
       'Private Property Structure', 'Homeless Shelter / Mission',
       'Schools During Un-Supervised Activity', 'Construction Site',
       'Police / Courts', 'Schools During Supervised Activity',
       'Religious Facilities', 'Universities / Colleges', 'Dealership',
       'Ttc Admin Or Support Facility', 'Pharmacy', 'Nursing Home',
       'Community Group Home', 'Halfway House'], dtype=object)

### Extract day of the week from occurence datetimes

Here, Monday is 0 and Sunday is 6

In [17]:
df['dayofweek'] = df.OCC_DATE.dt.dayofweek

### Trigonometric encoding for day of the week and occurence hour
These time based variables have a natural cyclic structure so we can use a trigonometric transformation to reflect this (instead of having a discontinuity at midnight, for example). We explore both the sine and cosine component.

In [18]:
# trig encode day of the week
df['dayofweek_sin'] = np.sin(2 * np.pi * df.dayofweek / 7)
df['dayofweek_cos'] = np.cos(2 * np.pi * df.dayofweek / 7)

# trig encode occurence hour
df['hour_sin'] = np.sin(2 * np.pi * df.OCC_HOUR / 24)
df['hour_cos'] = np.cos(2 * np.pi * df.OCC_HOUR / 24)

### Add Canadian Holiday variable
We suspect that Holidays could affect the frequency and type of crime

In [19]:
# Set of Canadian Holidays
ca_holidays = holidays.Canada()

# Check if Occurence date is on a Candian Holiday
df['isholiday'] = df.OCC_DATE.map(lambda x: x.date() in ca_holidays)

### Add feature to capture the interaction between time and location
Here, we add new features that capture the interaction effects between time (cos and sin transformed hour) and location (lat and long coordinates).

In [20]:
df['lat_hour_cos'] = df['LAT_WGS84'] * df['hour_cos']
df['lat_hour_sin'] = df['LAT_WGS84'] * df['hour_sin']
df['long_hour_cos'] = df['LONG_WGS84'] * df['hour_cos']
df['long_hour_sin'] = df['LONG_WGS84'] * df['hour_sin']

### Clustering using k-prototypes
Add a feature based on k-prototypes clustering which is suitable for a mix of numeric (i.e. GPS coordinates) and categorical (i.e. neighborhoods). This data driven feature can provide insight on the structure of the data, and on a high level, provide a low-cardinality way of representing the interaction between time and location. For example, directly multiplying the binary variables for hour (24) and neighborhood (158) for example, results in 3792 new dummy variables with questionable interpretability. Instead, we can simply cluster similar times and similar locations together.


### Export clean data

In [21]:
df.to_csv('data/clean_data.csv', index=False)